In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.preprocessing import image
from tensorflow.keras.layers import Dense, Flatten, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

In [2]:

# Update dataset paths to local system
train_dir = r"A:\finalmodel\training_set"
eval_dir = r"A:\finalmodel\test_set"

uniq_labels = sorted(os.listdir(train_dir))

In [3]:
# Helper function to load images
def load_images(directory):
    images = []
    labels = []
    for idx, label in enumerate(uniq_labels):
        label_dir = os.path.join(directory, label)
        if os.path.isdir(label_dir):  # Ensure it's a directory
            for file in os.listdir(label_dir):
                file_path = os.path.join(label_dir, file)
                img = cv2.resize(cv2.imread(file_path), (64, 64))
                images.append(img)
                labels.append(idx)
    return np.array(images), np.array(labels)

In [4]:
# Load training data
images, labels = load_images(train_dir)

# Load evaluation data (if available)
X_eval, y_eval = None, None
if uniq_labels == sorted(os.listdir(eval_dir)):
    X_eval, y_eval = load_images(eval_dir)

In [5]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, stratify=labels)

# Normalize Images
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
if X_eval is not None:
    X_eval = X_eval.astype('float32') / 255.0

# One-hot encoding labels
y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(uniq_labels))
y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(uniq_labels))
if y_eval is not None:
    y_eval = tf.keras.utils.to_categorical(y_eval, num_classes=len(uniq_labels))

In [6]:
# Load VGG16 Model
classifier_vgg16 = VGG16(input_shape=(64, 64, 3), include_top=False, weights='imagenet')
for layer in classifier_vgg16.layers:
    layer.trainable = False

# Modify VGG16 Model
classifier1 = Flatten()(classifier_vgg16.output)
classifier1 = Dense(units=256, activation='relu')(classifier1)
classifier1 = Dropout(0.6)(classifier1)
classifier1 = Dense(units=len(uniq_labels), activation='softmax')(classifier1)
model = Model(inputs=classifier_vgg16.input, outputs=classifier1)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# Train VGG16 Model
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

# Save Model
model.save('model_vgg16.h5')

# Evaluate Model
test_score = model.evaluate(X_test, y_test, verbose=0)
print(f"VGG16 Accuracy on Test Data: {round(test_score[1]*100, 3)}%")

if X_eval is not None:
    eval_score = model.evaluate(X_eval, y_eval, verbose=0)
    print(f"VGG16 Accuracy on Evaluation Data: {round(eval_score[1]*100, 3)}%")


Epoch 1/5
576/576 ━━━━━━━━━━━━━━━━━━━━ 317s 546ms/step - accuracy: 0.7982 - loss: 0.9432 - val_accuracy: 0.9721 - val_loss: 0.1065
Epoch 2/5
576/576 ━━━━━━━━━━━━━━━━━━━━ 304s 527ms/step - accuracy: 0.9647 - loss: 0.1368 - val_accuracy: 0.9811 - val_loss: 0.0695
Epoch 3/5
576/576 ━━━━━━━━━━━━━━━━━━━━ 312s 542ms/step - accuracy: 0.9740 - loss: 0.0901 - val_accuracy: 0.9818 - val_loss: 0.0565
Epoch 4/5
576/576 ━━━━━━━━━━━━━━━━━━━━ 300s 521ms/step - accuracy: 0.9775 - loss: 0.0743 - val_accuracy: 0.9861 - val_loss: 0.0447
Epoch 5/5
576/576 ━━━━━━━━━━━━━━━━━━━━ 305s 529ms/step - accuracy: 0.9806 - loss: 0.0645 - val_accuracy: 0.9885 - val_loss: 0.0388


VGG16 Accuracy on Test Data: 98.849%
VGG16 Accuracy on Evaluation Data: 99.444%
